# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Описание данных
Данные находятся в файле /datasets/autos.csv.

Признаки
**DateCrawled** — дата скачивания анкеты из базы

**VehicleType** — тип автомобильного кузова

**RegistrationYear** — год регистрации автомобиля

**Gearbox** — тип коробки передач

**Power** — мощность (л. с.)

**Model** — модель автомобиля

**Kilometer**  — пробег (км)

**RegistrationMonth** — месяц регистрации автомобиля

**FuelType** — тип топлива

**Brand** — марка автомобиля

**NotRepaired** — была машина в ремонте или нет

**DateCreated** — дата создания анкеты

**NumberOfPictures** — количество фотографий автомобиля

**PostalCode** — почтовый индекс владельца анкеты (пользователя)

**LastSeen** — дата последней активности пользователя

Целевой признак:
**Price** — цена (евро)

## Подготовка данных

Импортируем необходимые нам библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder 
from catboost import CatBoostRegressor
import time

Прочитаем наш датасет

In [2]:
data = pd.read_csv('/datasets/autos.csv')

In [3]:
data.head(20)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


Переведём столбцы из строчных в категориальные с помощью метода astype

In [5]:
data['VehicleType']=data['VehicleType'].astype('category')
data['Gearbox']=data['Gearbox'].astype('category')
data['Model']=data['Model'].astype('category')
data['FuelType']=data['FuelType'].astype('category')
data['Brand'] = data['Brand'].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null category
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null category
Power                354369 non-null int64
Model                334664 non-null category
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null category
Brand                354369 non-null category
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: category(5), int64(7), object(4)
memory usage: 31.8+ MB


Наблюдаем в пяти столбцах отсутствующие значения. 

In [6]:
data = data.dropna(subset =['VehicleType'])
data = data.dropna(subset =['Gearbox'])
data = data.dropna(subset =['Model'])
data = data.dropna(subset =['FuelType'])
data = data.dropna(subset =['NotRepaired'])
#data = data.dropna(subset =['Brand'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245814 entries, 3 to 354367
Data columns (total 16 columns):
DateCrawled          245814 non-null object
Price                245814 non-null int64
VehicleType          245814 non-null category
RegistrationYear     245814 non-null int64
Gearbox              245814 non-null category
Power                245814 non-null int64
Model                245814 non-null category
Kilometer            245814 non-null int64
RegistrationMonth    245814 non-null int64
FuelType             245814 non-null category
Brand                245814 non-null category
NotRepaired          245814 non-null object
DateCreated          245814 non-null object
NumberOfPictures     245814 non-null int64
PostalCode           245814 non-null int64
LastSeen             245814 non-null object
dtypes: category(5), int64(7), object(4)
memory usage: 23.9+ MB


In [8]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,245814.000000,245814.000000,245814.000000,245814.000000,245814.000000,245814.0,245814.000000
mean,5125.346717,2002.918699,119.970884,127296.716216,6.179701,0.0,51463.186002
std,4717.948673,6.163765,139.387116,37078.820368,3.479519,0.0,25838.058847
min,0.000000,1910.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1499.000000,1999.000000,75.000000,125000.000000,3.000000,0.0,30966.000000
50%,3500.000000,2003.000000,110.000000,150000.000000,6.000000,0.0,50769.000000
75%,7500.000000,2007.000000,150.000000,150000.000000,9.000000,0.0,72379.000000
max,20000.000000,2018.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Наблюдаем нулевые и аномальные значения. Сделаем срез, где их не наблюдается.

In [9]:
data1 = data.query('Price>1500 & RegistrationYear > 1990 & RegistrationMonth > 0 & Power > 75 & Power < 200' )
data1.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,126270.000000,126270.000000,126270.000000,126270.000000,126270.000000,126270.0,126270.000000
mean,6744.351216,2005.295739,130.636818,123914.627386,6.418833,0.0,52614.611689
std,4450.430292,4.647867,30.282040,38546.797422,3.347015,0.0,25878.734416
min,1506.000000,1991.000000,76.000000,5000.000000,1.000000,0.0,1067.000000
25%,3190.000000,2002.000000,105.000000,100000.000000,4.000000,0.0,32105.000000
50%,5500.000000,2006.000000,130.000000,150000.000000,6.000000,0.0,52146.000000
75%,9199.750000,2009.000000,150.000000,150000.000000,9.000000,0.0,73479.000000
max,20000.000000,2018.000000,199.000000,150000.000000,12.000000,0.0,99998.000000


In [10]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126270 entries, 6 to 354367
Data columns (total 16 columns):
DateCrawled          126270 non-null object
Price                126270 non-null int64
VehicleType          126270 non-null category
RegistrationYear     126270 non-null int64
Gearbox              126270 non-null category
Power                126270 non-null int64
Model                126270 non-null category
Kilometer            126270 non-null int64
RegistrationMonth    126270 non-null int64
FuelType             126270 non-null category
Brand                126270 non-null category
NotRepaired          126270 non-null object
DateCreated          126270 non-null object
NumberOfPictures     126270 non-null int64
PostalCode           126270 non-null int64
LastSeen             126270 non-null object
dtypes: category(5), int64(7), object(4)
memory usage: 12.3+ MB


Построим модель кэтбустинга на исходных данных

In [11]:
#def f(x):
#    return x+2
#print(f(5))

In [12]:
from sklearn.metrics import r2_score


In [13]:
features_train1, features_test1, target_train1, target_test1 = train_test_split(
    data1.drop(['Price','DateCrawled','PostalCode','NumberOfPictures',
              'DateCreated','LastSeen'], axis=1), data1.Price, test_size=0.25, random_state=12345)

categorial1 = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']


model1 =CatBoostRegressor(loss_function='RMSE',random_seed = 12345,iterations = 150,verbose=10)
#model1.fit(features_train1, target_train1, cat_features=cat_features)
model1.fit(features_train1, target_train1, cat_features=categorial1)

probabilities_test1 = model1.predict(features_test1)
#probabilities_one_valid1 = probabilities_valid1[:, 1]
mse = mean_squared_error(target_test1,probabilities_test1)
result = mse**0.5
#print(mse2 ,target_valid.values,pred_valid)
r2_score = r2_score(target_test1,probabilities_test1)



def model_time_cat (model, categorial, X_train, y_train, X_test):
    start = time.time()
    model.fit(X_train, y_train, cat_features=categorial)
    pred = model.predict(X_test)
    end = time.time()
    time_model = end-start
#    print(start, end, time_model)
    return pred,time_model

mdl=model_time_cat(model1,categorial1,features_train1,target_train1,features_test1)
#catboost = [mse,result,r2_score]
print("MSE =",mse)
print("RMSE = ",result)
print("R2 = ",r2_score)
print(model_time_cat)

0:	learn: 4367.9291966	total: 159ms	remaining: 23.6s
10:	learn: 3638.8987507	total: 1.75s	remaining: 22.2s
20:	learn: 3145.5586439	total: 3.34s	remaining: 20.5s
30:	learn: 2761.5590417	total: 4.93s	remaining: 18.9s
40:	learn: 2502.5950986	total: 6.52s	remaining: 17.3s
50:	learn: 2329.2195178	total: 8.02s	remaining: 15.6s
60:	learn: 2209.4527059	total: 9.51s	remaining: 13.9s
70:	learn: 2127.3850483	total: 11s	remaining: 12.2s
80:	learn: 2054.4805982	total: 12.5s	remaining: 10.6s
90:	learn: 2003.1515380	total: 13.9s	remaining: 9.01s
100:	learn: 1966.8699776	total: 15.4s	remaining: 7.46s
110:	learn: 1939.2818000	total: 16.8s	remaining: 5.89s
120:	learn: 1916.0747581	total: 18.2s	remaining: 4.35s
130:	learn: 1898.3651970	total: 19.6s	remaining: 2.84s
140:	learn: 1883.3441008	total: 21s	remaining: 1.34s
149:	learn: 1871.4430344	total: 22.3s	remaining: 0us
0:	learn: 4367.9291966	total: 33.7ms	remaining: 5.02s
10:	learn: 3638.8987507	total: 1.62s	remaining: 20.5s
20:	learn: 3145.5586439	total

In [14]:
mdl[1]

23.91545534133911

Создаём табличку для фиксирования результатов

In [15]:
a = {'Метрика': ['MSE','RMSE','R2-score','time']}
total_table = pd.DataFrame(data = a)

In [16]:
col = 'CatBoost'
total_table.loc[0,col] = mse
total_table.loc[1,col]= result
total_table.loc[2,col] = r2_score
total_table.loc[3,col] = mdl[1]

Разберёмся с категориальными признаками с помощью структуры данных OrdinalEncoder

In [17]:
encoder = OrdinalEncoder() 
data2 =  data1[['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']]
#data2.head(10)
  #          'Model',    'FuelType', 'Brand', 'NotRepaired']]
data_ordinal = pd.DataFrame(encoder.fit_transform(data2), 
                                                      columns=data2.columns) 
data_ordinal.head()
#print(data2)

,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,1.0,1.0,7.0,6.0,25.0,0.0
1,4.0,1.0,9.0,6.0,19.0,0.0
2,7.0,1.0,157.0,2.0,35.0,1.0
3,6.0,1.0,146.0,2.0,23.0,0.0
4,0.0,1.0,56.0,2.0,10.0,0.0


## Обучение моделей

Обучим разные модели - линейную регресси и случайный лес.
Для начала выделим количественные признаки

In [18]:
data_count = data1[['RegistrationYear','Power','Kilometer','RegistrationMonth','Price']]
data_final = [data_ordinal, data_count]
print(data_final)

[        VehicleType  Gearbox  Model  FuelType  Brand  NotRepaired
0               1.0      1.0    7.0       6.0   25.0          0.0
1               4.0      1.0    9.0       6.0   19.0          0.0
2               7.0      1.0  157.0       2.0   35.0          1.0
3               6.0      1.0  146.0       2.0   23.0          0.0
4               0.0      1.0   56.0       2.0   10.0          0.0
...             ...      ...    ...       ...    ...          ...
126265          4.0      0.0   88.0       2.0   20.0          0.0
126266          4.0      1.0   25.0       6.0    1.0          0.0
126267          4.0      1.0  130.0       2.0   29.0          0.0
126268          4.0      1.0  107.0       2.0   35.0          0.0
126269          0.0      1.0  204.0       2.0   35.0          0.0

[126270 rows x 6 columns],         RegistrationYear  Power  Kilometer  RegistrationMonth  Price
6                   2004    109     150000                  8   2200
10                  2004    105     15000

In [19]:
features_train, features_test, target_train, target_test = train_test_split(
    data_count.drop(['Price'], axis=1),data_count.Price, test_size=0.25, random_state=12345)    

Построим модель линейной регрессии и случайного леса.

In [20]:
#model = lgb.LGBMRegressor(n_splits = 10,n_repeats = 3,random_state = 12345)
#model.fit(features_train, target_train)
#pred_valid_light = model.predict(features_valid)

#mse_light = mean_squared_error(target_valid,pred_valid_light)
#result_light = mse_light**0.5
#r2_score_light = r2_score(target_valid,pred_valid_light)

#print("MSE =",mse_light)
#print("RMSE = ",result_light)
#print("R2 = ",r2_score_light)

Далее проверим нашу модель на линейной регрессии.

In [21]:
from sklearn.metrics import r2_score

In [22]:
model2 =  LinearRegression()
model2.fit(features_train,target_train) 
pred_test = model2.predict(features_test)

def model_time_linear(model, X_train, y_train, X_test):
    start = time.time()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    end = time.time()
    time_model = end-start
#    print(start, end, time_model)
    return pred,time_model

mdl1=model_time_linear(model2,features_train,target_train,features_test)

mse1 = mean_squared_error(target_test,pred_test)
result1 = mse**0.5
r2_score1 = r2_score(target_test,pred_test)
mdl2 = mdl1[1]

print("MSE =",mse1)
print("RMSE = ",result1)
print("R2 = ",r2_score1)

MSE = 7293691.141800072
RMSE =  1832.095463075788
R2 =  0.6288183607531617


Запишем данные в таблицу

In [23]:
col = 'LinearRegression'
total_table.loc[0,col] = mse1
total_table.loc[1,col]= result1
total_table.loc[2,col] = r2_score1
total_table.loc[3,col] = mdl2

Посмотрим на поведение модели на "случайном лесу".

In [24]:
from sklearn.metrics import r2_score

In [25]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10,11):
    for depth in range (1,2):
        model3 =   RandomForestRegressor(random_state=12345,n_estimators=est, max_depth= depth)
        model3.fit(features_train,target_train) 
        pred_test = model3.predict(features_test) 
        mse2 = mean_squared_error(target_test,pred_test)
        rmse2 = mse2**0.5
        r2_score2 = r2_score(target_test,pred_test)
        if result < best_result:
            best_model = model3
            best_result = result
            best_est = est
            best_depth = depth


def model_time_forest(model, X_train, y_train, X_test):
    start = time.time()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    end = time.time()
    time_model = end-start
#   print(start, end, time_model)
    return pred,time_model       

mdl2 = model_time_forest(model3,features_train,target_train,features_test)
mdl3 = mdl2[1]           
print("Среднеквадратичная ошибка:",mse2,";" ,"Количество деревьев:", best_est, ";",\
      "Максимальная глубина:", depth,";","Среднеквадратичное отклонение:",rmse2
      ,"Метрика R-2:",r2_score2)


Среднеквадратичная ошибка: 10748913.8588919 ; Количество деревьев: 10 ; Максимальная глубина: 1 ; Среднеквадратичное отклонение: 3278.5536230008347 Метрика R-2: 0.45297937783503717


Обновим таблицу

In [26]:
col = 'RandomForest'
total_table.loc[0,col] = mse2
total_table.loc[1,col]= rmse2
total_table.loc[2,col] = r2_score2
total_table.loc[3,col] = mdl3

Выведем всю таблицу:

In [27]:
total_table

,Метрика,CatBoost,LinearRegression,RandomForest
0,MSE,3.356574e+06,7.293691e+06,1.074891e+07
1,RMSE,1.832095e+03,1.832095e+03,3.278554e+03
2,R2-score,8.291813e-01,6.288184e-01,4.529794e-01
3,time,2.391546e+01,3.999424e-02,1.478927e-01


## Анализ моделей

В среднем меньше всего ошибка в модели "CatBoost" - в двух метриках из трёх, метрика R-2 меньше всего в модели "Случайного леса". Но у "леса" самые большие значения метрик "MSE" и "RMSE",но "CatBoost" не может быть лучшей моделью, потому что время на подсчёта модели на два-три порядка больше остальных моделей. Поэтому, "Линейная Регрессия" - лучшая модель.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей